# Langchain
* [Github repo](https://github.com/hwchase17/langchain)
* [Python documentation](https://python.langchain.com/en/latest/)


* See https://beta.openai.com/account/api-keys to get your API key
* Follow https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety to set up the OPENAI_API_KEY environment variable (used in the code below)


In [1]:
import os
openai_api_key = os.environ["OPENAI_API_KEY"]

## Using Azure Open AI API

* https://python.langchain.com/en/latest/modules/models/llms/integrations/azure_openai_example.html

## LLM models
LangChain provides a consistent interface to large language models (various types, various providers, etc.)

### 1. Completion API
* E.g. text-davinci-003
* Mode = text-in-text-out

In [2]:
# Completion API (text-davinci-003): text-in-text-out

from langchain.llms import OpenAI

llm = OpenAI(model_name='text-davinci-003',
             temperature=0.9,
             openai_api_key=openai_api_key,
             max_tokens=1024,
             top_p=1)

prompt = "What is a dish associated with Pune?"

response = llm(prompt)

print(response)



A dish associated with Pune is Paani Puri, a type of chaat (a savory snack) consisting of fried, hollow puri (an unleavened deep-fried Indian bread) filled with potatoes, onions, chutney and a spicy flavored water (paani).


### 2. Chat API
* E.g. gpt-3.5-turbo
* Mode = list-of-messages-in-message-out

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(model_name='gpt-3.5-turbo',
                  temperature=0.9,
                  openai_api_key=openai_api_key)

response = chat(
    [
        SystemMessage(content="You are an AI assistant that helps a user learn about food. Keep responses short and under 2 sentences."),
        HumanMessage(content="What is a dish associated with Pune?")
    ]
)

response

AIMessage(content='Misal Pav is a popular dish associated with Pune, Maharashtra.', additional_kwargs={})

### 3. Embeddings
* E.g. text-embedding-ada-002
* Mode = text-in-vector-out

In [73]:
from langchain.embeddings import OpenAIEmbeddings

openai_embed = OpenAIEmbeddings(model='text-embedding-ada-002',
                                openai_api_key=openai_api_key)

query_embeddings = openai_embed.embed_query("Vada Pav") 

print(type(query_embeddings))
print(len(query_embeddings))
print(query_embeddings[:5])

<class 'list'>
1536
[0.007298551898297813, -0.010412962080020289, 0.011232186936721102, -0.0041028963955430245, -0.00617804139751549]


## PromptTemplate
Allows creating the final prompt from a template using user input and other non-static information.

In [47]:
from langchain.prompts import PromptTemplate

# Define a prompt template with placeholders for input variables.
prompt_template = PromptTemplate(
        template = 'What is a {reco_type} most associated with {location}?',
        input_variables = ['reco_type', 'location']
)

# Create the final prompt by filling in the defined variables.
final_prompt = prompt_template.format(reco_type='dish',  location='Pune')
print("FINAL PROMPT: " + final_prompt)

response = llm(final_prompt) 
print(response)

FINAL PROMPT: What is a dish most associated with Pune?


Pav Bhaji is a popular dish from Pune. The dish is a combination of mashed vegetables, potatoes and spices served with a lightly toasted bun (or pav).


In [48]:
print(llm(prompt_template.format(reco_type='dish', location='Hyderabad')))



Hyderabadi Biryani is the most popular and iconic dish associated with Hyderabad. It is a unique twist on traditional Indian biryani, and is a combination of spices, rice, yogurt, and either chicken or lamb.


In [56]:
print(llm(prompt_template.format(reco_type='movie', location='Hyderabad')))
print(llm(prompt_template.format(reco_type='place to visit', location='Hyderabad')))
print(llm(prompt_template.format(reco_type='phrase', location='Hyderabad')))



The most popular movie associated with Hyderabad is Baahubali: The Beginning, an Indian epic fantasy film.


"Pearl of the Deccan".


## Structured Output Parser

In [57]:
# 1. Define the schema

from langchain.output_parsers import StructuredOutputParser, ResponseSchema

response_schemas = [
    ResponseSchema(name="dish_name", description="This is the name of the dish"),
    ResponseSchema(name="dish_description", description="This is a short description of the dish")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [58]:
# 2. Generates output format instructions which can be included in your prompt.

format_instructions = output_parser.get_format_instructions()
print (format_instructions)

The output should be a markdown code snippet formatted in the following schema:

```json
{
	"dish_name": string  // This is the name of the dish
	"dish_description": string  // This is a short description of the dish
}
```


In [26]:
# 3. Incorporate into final prompt.

template = '''
You will be given a location by the user.
Please provide a dish name and a short description of the dish associated with the location.

{format_instructions}

% LOCATION:
{location}

YOUR RESPONSE:
'''

prompt_template = PromptTemplate(
    input_variables=["location"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

prompt = prompt_template.format(location="Pune")
print('FINAL PROMPT:')
print(prompt)

FINAL PROMPT:

You will be given a location by the user.
Please provide a dish name and a short description of the dish associated with the location.

The output should be a markdown code snippet formatted in the following schema:

```json
{
	"dish_name": string  // This is the name of the dish
	"dish_description": string  // This is a short description of the dish
}
```

% LOCATION:
Pune

YOUR RESPONSE:



In [59]:
# 4. LLM now outputs an easily parsable response.

response = llm(prompt)
print(response)

```json
{
	"dish_name": "Pav Bhaji",
	"dish_description": "Pav Bhaji is a popular fast food dish originating from the Indian city of Mumbai. It consists of a spicy potato and vegetable curry served with buttered rolls (pav) and a variety of traditional accompaniments."
}
```


In [60]:
# 5. Output parser can extract named fields in the schema.

output_parser.parse(response)['dish_name']

'Pav Bhaji'

## Chains
Create chains (workflows) by running multiple components

### 1. Simple sequential chains

In [61]:
# Chain component 1: Get dish for location.

from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain

template1 = """You will be given a location from the user. Please provide a dish associated with the location.

% LOCATION
{location}

YOUR RESPONSE:
"""
prompt_template1 = PromptTemplate(input_variables=["location"], template=template1)

# LLMChain is a simple chain which takes a prompt template, formats with user input and returns LLM output.
location_dish_chain = LLMChain(llm=llm, prompt=prompt_template1)

In [62]:
# Chain component 2: Get recipe for a dish.

template2 = """Given the name of a dish, give a short recipe for making it at home.

% DISH
{dish}

YOUR RESPONSE:
"""
prompt_template2 = PromptTemplate(input_variables=["dish"], template=template2)

recipe_chain = LLMChain(llm=llm, prompt=prompt_template2)

In [63]:
overall_chain = SimpleSequentialChain(
    chains=[location_dish_chain, recipe_chain],
    verbose=True)
chain_output = overall_chain.run("Pune")



> Entering new SimpleSequentialChain chain...
Pav Bhaji

Ingredients: 
- 2 potatoes, diced
- 2 cups cauliflower, diced 
- 1 onion, diced 
- 1 green bell pepper, diced 
- 2 tablespoons vegetable oil 
- 2 teaspoons cumin seeds 
- 2 teaspoons mustard seeds 
- 1 teaspoon turmeric powder 
- 1 teaspoon garam masala 
- 2 tablespoons tomato paste 
- 1 cup cooked peas 
- Salt, to taste 
- 2 tablespoons butter 
- 1 cup cooked sweet corn 
- 8-10 pav buns 
- Chopped cilantro

Instructions: 
1. Heat oil in a large pan over medium heat. Add cumin and mustard seeds and let it splutter. 
2. Add the onions and sauté until they are golden brown. 
3. Add the potatoes, cauliflower, green bell pepper and sauté for another 2-3 minutes. 
4. Add the turmeric powder, garam masala and tomato paste and mix well. 
5. Add the cooked peas, salt and about ½ cup of water to cover all the vegetables and cover it with a lid. Cook until the vegetables are cooked through. 
6. Add the butter and sweet corn and mix every

The chain output will return the response for the final component.

In [64]:
print(chain_output)


Ingredients: 
- 2 potatoes, diced
- 2 cups cauliflower, diced 
- 1 onion, diced 
- 1 green bell pepper, diced 
- 2 tablespoons vegetable oil 
- 2 teaspoons cumin seeds 
- 2 teaspoons mustard seeds 
- 1 teaspoon turmeric powder 
- 1 teaspoon garam masala 
- 2 tablespoons tomato paste 
- 1 cup cooked peas 
- Salt, to taste 
- 2 tablespoons butter 
- 1 cup cooked sweet corn 
- 8-10 pav buns 
- Chopped cilantro

Instructions: 
1. Heat oil in a large pan over medium heat. Add cumin and mustard seeds and let it splutter. 
2. Add the onions and sauté until they are golden brown. 
3. Add the potatoes, cauliflower, green bell pepper and sauté for another 2-3 minutes. 
4. Add the turmeric powder, garam masala and tomato paste and mix well. 
5. Add the cooked peas, salt and about ½ cup of water to cover all the vegetables and cover it with a lid. Cook until the vegetables are cooked through. 
6. Add the butter and sweet corn and mix everything together. Cook for another 2-3 minutes. 
7. Take off

## Basic Q&A workflow using Document loaders, Text splitters, RetrievalQA
* The code below uses open AI embeddings and chromadb (pip install chromadb, VC++ build tools needed), but langchain supports many other [vector stores](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html). Feel free to explore!

In [106]:
from langchain.document_loaders import GutenbergLoader

# Load a long document.
loader = GutenbergLoader('https://www.gutenberg.org/cache/epub/1515/pg1515.txt')
doc = loader.load()

print(doc[0].page_content[:80].rstrip())
print('Document length in characters:', len(doc[0].page_content))

The Project Gutenberg eBook of The Merchant of Venice, by William Shakespeare
Document length in characters: 154211


In [50]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

# We need to split long documents into smaller chunks to stay under the LLM token limit.
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(doc[0].page_content)

In [55]:
texts[2]

'THE PRINCE OF MOROCCO, suitor to Portia\r\n\n\nTHE PRINCE OF ARRAGON, suitor to Portia\r\n\n\nANTONIO, a merchant of Venice\r\n\n\nBASSANIO, his friend, suitor to Portia\r\n\n\nGRATIANO, friend to Antonio and Bassanio\r\n\n\nSOLANIO, friend to Antonio and Bassanio\r\n\n\nSALARINO, friend to Antonio and Bassanio\r\n\n\nLORENZO, in love with Jessica\r\n\n\nSHYLOCK, a rich Jew\r\n\n\nTUBAL, a Jew, his friend\r\n\n\nLAUNCELET GOBBO, a clown, servant to Shylock\r\n\n\nOLD GOBBO, father to Launcelet\r\n\n\nLEONARDO, servant to Bassanio\r\n\n\nBALTHAZAR, servant to Portia\r\n\n\nSTEPHANO, servant to Portia\r\n\n\nSALERIO, a messenger from Venice\r\n\n\n\r\n\n\nPORTIA, a rich heiress\r\n\n\nNERISSA, her waiting-woman\r\n\n\nJESSICA, daughter to Shylock\r\n\n\n\r\n\n\nMagnificoes of Venice, Officers of the Court of Justice, a Gaoler,\r\n\n\nServants and other Attendants\r\n\n\n\r\n\n\nSCENE: Partly at Venice, and partly at Belmont, the seat of Portia on\r\n\n\nthe Continent\r\n\n\n\r\n\n\n\r\n

In [77]:
# Create embeddings for each passage for retrieval.
embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": f"{i}"} for i in range(len(texts))])

Using embedded DuckDB without persistence: data will be transient


In [96]:
from langchain.chains import RetrievalQAWithSourcesChain
retriever = docsearch.as_retriever(search_type='similarity', search_kwargs={'k':2})
chain = RetrievalQAWithSourcesChain.from_chain_type(OpenAI(temperature=0), chain_type="stuff", retriever=retriever)
response = chain({"question": "From what character flaw does Bassanio believe Gratiano suffers?"}, return_only_outputs=True)
response

{'answer': ' Bassanio believes Gratiano suffers from a character flaw of being too wild, rude, and bold of voice.\n',
 'sources': '40, 103'}

In [103]:
def print_answer_with_source(response):
    print('%ANSWER%\n\n' + response['answer'])
    source_ids = [int(x) for x in response['sources'].split(', ')]
    print('%SOURCE%\n\n' + texts[source_ids[0]].replace('\n\n', '\n').replace('\n\n', '\n'))

print_answer_with_source(response)

%ANSWER%

 Bassanio believes Gratiano suffers from a character flaw of being too wild, rude, and bold of voice.

%SOURCE%

You have obtain’d it.

GRATIANO.
You must not deny me, I must go with you to Belmont.

BASSANIO.
Why, then you must. But hear thee, Gratiano,
Thou art too wild, too rude, and bold of voice,
Parts that become thee happily enough,
And in such eyes as ours appear not faults;
But where thou art not known, why there they show
Something too liberal. Pray thee, take pain
To allay with some cold drops of modesty
Thy skipping spirit, lest through thy wild behaviour
I be misconst’red in the place I go to,
And lose my hopes.

GRATIANO.
Signior Bassanio, hear me.
If I do not put on a sober habit,
Talk with respect, and swear but now and then,
Wear prayer-books in my pocket, look demurely,
Nay more, while grace is saying, hood mine eyes
Thus with my hat, and sigh, and say “amen”;
Use all the observance of civility
Like one well studied in a sad ostent
To please his grandam, nev

## Agents

Agents
* Some applications will require not just a predetermined chain of calls to LLMs/other tools, but **potentially an unknown chain that depends on the user’s input**. In these types of chains, there is a “agent” which has access to a suite of tools. Depending on the user input, the agent can then decide which, if any, of these tools to call.

Tools
* Langchain has wrappers for many tools - see the full list [here](https://python.langchain.com/en/latest/modules/agents/tools.html)


In [33]:
from langchain.utilities import BingSearchAPIWrapper, WikipediaAPIWrapper, PythonREPL

# Bing API usage requires the BING_SUBSCRIPTION_KEY and BING_SEARCH_URL environment variables to be present.
bingapi = BingSearchAPIWrapper(k=1)
wikipedia = WikipediaAPIWrapper(top_k_results=1)
python_repl = PythonREPL()

In [30]:
from langchain import OpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

In [38]:
tools = [
    Tool(
        name="BingSearch",
        func=bingapi.run,
        description="Used to search the web with a query"
    ),
    Tool(
        name="Wikipedia",
        func=wikipedia.run,
        description="A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, historical events, or other subjects. Input should be a search query."
    ),
    Tool(
        name="PythonREPL",
        func=python_repl.run,
        description="Used to run python code for performing complex calculations"
    )
]

llm = OpenAI(temperature=0, model_name="text-davinci-003")
react = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [41]:
react.run("How many minutes has it been since the Matrix was released?")



> Entering new AgentExecutor chain...
 I need to find out when the Matrix was released and then calculate the difference in time.
Action: BingSearch
Action Input: "When was the Matrix released"
Observation: The <b>Matrix</b> sold more than 107,000 DVD copies in just two weeks, breaking Armageddon &#39; s record for becoming the country&#39;s best-selling DVD title. The Ultimate <b>Matrix</b> Collection was <b>released</b> on HD DVD on May 22, 2007 and on Blu-ray on October 14, 2008.
Thought: I need to find the exact date the Matrix was released.
Action: Wikipedia
Action Input: "The Matrix"
Observation: Page: The Matrix
Summary: The Matrix is a 1999 science fiction action film written and directed by the Wachowskis. It is the first installment in the Matrix film series, starring Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, Hugo Weaving, and Joe Pantoliano, and depicts a dystopian future in which humanity is unknowingly trapped inside the Matrix, a simulated reality that intelli

'The Matrix was released on March 31, 1999, which means it has been approximately 6,822,400 minutes since its release.'